In [1]:
import zapimoveis_scraper as zap
import requests as r
import io
import pandas as pd
import unidecode as code
import time

In [2]:
#Importando dados dos bairros do Brasil
df_bairros_municipios = pd.read_csv("Bairros.csv - Bairro.xlsx - Sheet 1.csv",error_bad_lines=False, sep=",")
df_bairros_municipios.drop(labels="Endereço", axis=1, inplace=True)

/home/abel/Documentos/Github/WebScraping-projects/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [44]:
#Retornando no formato lista
list_municípios = df_bairros_municipios['Cidade'].to_list()
list_bairros = df_bairros_municipios['Bairros'].to_list()

#Realizando o enconding para realizar a pesquisa
list_municipios = [code.unidecode(cidade.replace(" ", "-").lower()) for cidade in list_municípios]
list_bairros = [code.unidecode(bairro.lower().replace(" ", "-")) for bairro in list_bairros]

#Unificando listas em um único data-frame
df_bairros_municipios = pd.DataFrame(data = zip(list_municipios,list_bairros))

In [45]:
#Municípios
rondonia = df_bairros_municipios.iloc[0:196,:]
amazonas = df_bairros_municipios.iloc[197:475,:]
roraima = df_bairros_municipios.iloc[476:530,:]
para = df_bairros_municipios.iloc[531:777,:]
amapa = df_bairros_municipios.iloc[778:818,:]
#Acre
#Distrito Federal
maranhao = df_bairros_municipios.iloc[819:859,:]
piaui = df_bairros_municipios.iloc[860:1240,:]
ceara = df_bairros_municipios.iloc[1241:2325,:]
rio_grande_do_norte = df_bairros_municipios.iloc[2326:2491,:]
paraiba = df_bairros_municipios.iloc[2492:2659,:]
pernambuco = df_bairros_municipios.iloc[2660:3186,:]
alagoas = df_bairros_municipios.iloc[3186:3308,:]
sergipe = df_bairros_municipios.iloc[3309:3396,:]
bahia = df_bairros_municipios.iloc[3397:3479,:]
minas_gerais = df_bairros_municipios.iloc[3480:5163,:]
espirito_santo = df_bairros_municipios.iloc[5164:5795,:]
rio_de_janeiro = df_bairros_municipios.iloc[5796:7039,:]
sao_paulo = df_bairros_municipios.iloc[7040:9016,:]
parana = df_bairros_municipios.iloc[9017:9772,:]
santa_catarina = df_bairros_municipios.iloc[9773:11219,:]
rio_grande_do_sul = df_bairros_municipios.iloc[11220:13199,:]
mato_grosso_do_sul = df_bairros_municipios.iloc[13200:13471,:]
mato_grosso = df_bairros_municipios.iloc[13472:14290,:]
goias = df_bairros_municipios.iloc[14291:14322,:]


In [46]:
list_bathrooms = []
list_value_rent = []
list_bedrooms = []
list_area_total = []
list_address = []
list_links = []
list_description = []

In [ ]:
list_sc = []
for index,linha in roraima.iterrows():
    list_sc.append(f"{linha[0]}++{linha[1]}")
list_sc

# Teste com a região de atibaia

In [50]:
lista_filtrada = list(filter(lambda a: a != "Não encontrado",list_anuncios))

In [ ]:
lista_filtrada

In [52]:
cont = 0
for item in lista_filtrada:
    cont+= len(item)
print(cont)

369


In [53]:
anuncios = zap.search(localization="sp+atibaia++caetetuba", num_pages=10)
print(anuncios, len(anuncios))

[<zapimoveis_scraper.item.ZapItem object at 0x7f928b3545e0>] 1


In [54]:
for atributo in anuncios:
    list_bathrooms.append(atributo.bathrooms)
    list_bedrooms.append(atributo.bedrooms)
    list_value_rent.append(atributo.price)
    list_area_total.append(atributo.total_area_m2)
    list_address.append(atributo.address)
    list_links.append(atributo.link)
    list_description.append(atributo.description)

# Colocando em um data frame
dicionario = {"Valor Aluguel": list_value_rent, 
              "Qtd Banheiros": list_bathrooms, 
              "Qtd Quartos": list_bedrooms, 
              "Area total" : list_area_total,
              "Endereço": list_address,
              "Link do Anúncio": list_links,
              "Descrição do Anúncio": list_description}

df_anuncios_casas_aluguel = pd.DataFrame.from_dict(dicionario, orient='index')
df_anuncios_casas_aluguel = df_anuncios_casas_aluguel.transpose()
 

# Exportando para um arquivo em csv
df_anuncios_casas_aluguel.to_csv("Anúncios de aluguel de casas.csv", sep=";", index=False)

In [3]:
import requests as r
from bs4 import BeautifulSoup
from selenium import webdriver as web
import numpy as np

In [4]:
#Retornando no formato lista
list_municípios = df_bairros_municipios['Cidade'].to_list()
list_estados = df_bairros_municipios['UF'].to_list()
#Realizando o enconding para realizar a pesquisa
list_municipios = [code.unidecode(cidade.replace(" ", "-")) for cidade in list_municípios]


#Unificando listas em um único data-frame
df_bairros = pd.DataFrame(data = zip(list_municipios,list_estados))
df_bairros.to_csv("municipios e estados", sep=";")

In [5]:
list_estados = df_bairros_municipios['UF'].unique()
list_estados = np.delete(list_estados,2)
lista_estados = list(np.insert(list_estados, 0,["TO","AC", "DF"]))

In [8]:
option = web.ChromeOptions()
#option.headless = True
driver = web.Chrome(options=option)

lista_cidades_site = []
list_ufs = []
list_urls = []
for estado in lista_estados:
    driver.get(f"https://ceps.io/{estado}/")
    time.sleep(5)
    list_cidades = driver.find_element_by_class_name("related")
    html__content = list_cidades.get_attribute('outerHTML')
    soup = BeautifulSoup(html__content, 'html.parser')
    soup_cities = soup.find_all("h4")
    soup_cities = soup_cities[:-27]
    links = [a['href'] for a in soup.find_all('a', href=True)]
    links = links[:-27]
    for cidade in soup_cities:
        lista_cidades_site.append(cidade.get_text())
        list_ufs.append(estado)
    for link in links:
        list_urls.append(link)
        
driver.quit()

/tmp/ipykernel_1924/2062776933.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  list_cidades = driver.find_element_by_class_name("related")


In [15]:
df_uf_cidade = pd.DataFrame(data=zip(lista_cidades_site,list_ufs,new_urls))

In [25]:
df_uf_cidade

,0,1,2
0,Abreulândia,TO,https://ceps.io/TO/Abreulandia/
1,Aguiarnópolis,TO,https://ceps.io/TO/Aguiarnopolis/
2,Aliança do Tocantins,TO,https://ceps.io/TO/Alianca-do-Tocantins/
3,Almas,TO,https://ceps.io/TO/Almas/
4,Alvorada,TO,https://ceps.io/TO/Alvorada/
...,...,...,...
10571,Vianópolis,GO,https://ceps.io/GO/Vianopolis/
10572,Vicentinópolis,GO,https://ceps.io/GO/Vicentinopolis/
10573,Vila Boa,GO,https://ceps.io/GO/Vila-Boa/
10574,Vila Propício,GO,https://ceps.io/GO/Vila-Propicio/


In [17]:
df_uf_cidade.to_csv("uf_cidades.csv")

## Pegando todos os bairros de todas as 10k de cidades no Brasil

In [ ]:
new_urls = []
for urls in list_urls:
    new_urls.append(f"https://ceps.io{urls}")
new_urls

In [ ]:
option = web.ChromeOptions()
#option.headless = True
driver = web.Chrome(options=option)

lista_bairros = []
for bairro in new_urls:
    driver.get(bairro)
    time.sleep(20)
    bairros = driver.find_element_by_class_name("related")
    html__content = bairros.get_attribute('outerHTML')
    soup = BeautifulSoup(html__content, 'html.parser')
    print(soup)
driver.quit()

In [43]:
option = web.ChromeOptions()
#option.headless = True
driver = web.Chrome(options=option)

ufs = []
lista_bairros = []
cities = []
for index, row in df_uf_cidade.iterrows():
    driver.get(row[2])
    time.sleep(5)
    list_bairros = driver.find_element_by_class_name("related")
    html__content = list_bairros.get_attribute('outerHTML')
    soup = BeautifulSoup(html__content, 'html.parser')
    soup_bairros = soup.find_all("h4")
    for bairro in soup_bairros:
        if bairro not in df_uf_cidade[0].to_list():
            lista_bairros.append(bairro.get_text())
            ufs.append(row[1])
            cities.append(row[0])
driver.quit()

/tmp/ipykernel_1924/580341197.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  list_bairros = driver.find_element_by_class_name("related")


In [44]:
df_bairros_cities_uf = pd.DataFrame(data = zip(ufs,cities,lista_bairros))
df_bairros_cities_uf.to_csv("Bairros Brasil")

In [47]:
df_bairros_cities_uf.rename(columns={0: "UF", 1:"Cidade", 2:"Bairro"}, inplace=True)

In [49]:
df_bairros_cities_uf.to_csv("Bairros-Brasil.csv")